In [100]:
using DataFrames

# To run this notebook, you need to have a data file available.
# You can either run the phenotype preprocessing scripts in ../preprocessing directory
# Or scp -r username@sherlock.stanford.edu:/scratch/PI/dpwall/DATA/phenotypes/jsonschema ../data

# Data
df = readtable("../data/all_samples_filtered_train.csv", header=false)

m, n = size(df)

(16291, 250)

In [101]:
# Form sparse array
all_data = sparse(Array(df))
dropzeros!(all_data)
p = size(nonzeros(all_data), 1)

1873895

In [102]:
adir_indices = 1:139
ados_indices = 140:185
srs_indices = 186:250

186:250

In [103]:
using LowRankModels
losses = Array{Loss}(n)
for i=1:n
    options = unique(all_data[:, i])
    if length(options) == 3
        losses[i] = HingeLoss()
    else
        losses[i] = OrdinalHingeLoss(-1, maximum(options))
    end
end

In [95]:
maximum(all_data), minimum(all_data)

(4, -1)

In [86]:
#Pkg.update("LowRankModels")

In [ ]:
using LowRankModels

k=10

#losses = LogisticLoss()
#losses = QuadLoss()
#losses = OrdinalHingeLoss(4)
rx = OneReg()
ry = QuadReg()

glrm = GLRM(all_data, losses, rx, ry, k);

X,Y,ch = fit!(glrm, SparseProxGradParams(max_iter=500), verbose=true); # fit GLRM

LowRankModels.SparseProxGradParams(1.0, 500, 1, 1.0e-5, 0.01)
Fitting GLRM


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at ./<missing>:0
 [5] #objective#61 at /Users/kelley/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:82 [inlined]
 [6] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM) at ./<missing>:0
 [7] #fit!#85(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/sparse_proxgrad.jl:50
 [8] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at ./<missing>:0
 [9] 

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] sumabs2(::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}) at ./deprecated.jl:57
 [3] evaluate(::LowRankModels.QuadReg, ::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}) at /Users/kelley/.julia/v0.6/LowRankModels/src/regularizers.jl:57
 [4] #calc_penalty#62(::Array{Union{Int64, Range{Int64}},1}, ::Function, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at /Users/kelley/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:100
 [5] (::LowRankModels.#kw##calc_penalty)(::Array{Any,1}, ::LowRankModels.#calc_penalty, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at ./<missing>:0
 [6] #objective#60(::Bool, ::Bool, ::Array{Union{Int64, Range{Int64}},1}, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at /Users/kelley/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:72
 [7] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective

obj went up to 2.1217373190101967e6; reducing step size to 0.8508543750000003
Iteration 10: objective value = 1.8985780319383182e6
obj went up to 1.8419791491014797e6; reducing step size to 0.8380662854769941
obj went up to 1.7521798382461036e6; reducing step size to 0.6791165386077139
Iteration 20: objective value = 1.7474334150222358e6
obj went up to 1.7108947598840625e6; reducing step size to 0.5503135981040285
Iteration 30: objective value = 1.6878036199240435e6
obj went up to 1.6879922686605186e6; reducing step size to 0.4916485691972446
obj went up to 1.671422794252496e6; reducing step size to 0.43923740286716423
Iteration 40: objective value = 1.6628814182514285e6
obj went up to 1.6580480064635242e6; reducing step size to 0.35593053894587073
Iteration 50: objective value = 1.6496687012828153e6
obj went up to 1.6495939764063163e6; reducing step size to 0.35058100825090116
obj went up to 1.6410964863384028e6; reducing step size to 0.2840889377735147
Iteration 60: objective value =

In [9]:
using MLBase

approx = X.'*Y
i_values = approx[validate_data .!= 0]
a_values = validate_data[validate_data .!= 0]
diff = i_values - a_values
println(sum((abs.(diff) .< 0.5) .& (a_values .== 1))/sum(a_values .== 1), ", ", sum((abs.(diff) .< 1.5) .& (a_values .== 1))/sum(a_values .== 1))
println(sum((abs.(diff) .< 0.5) .& (a_values .== 2))/sum(a_values .== 2), ", ", sum((abs.(diff) .< 1.5) .& (a_values .== 2))/sum(a_values .== 2))
println(sum((abs.(diff) .< 0.5) .& (a_values .== 3))/sum(a_values .== 3), ", ", sum((abs.(diff) .< 1.5) .& (a_values .== 3))/sum(a_values .== 3))
println(sum((abs.(diff) .< 0.5) .& (a_values .== 4))/sum(a_values .== 4), ", ", sum((abs.(diff) .< 1.5) .& (a_values .== 4))/sum(a_values .== 4))

i_round_values = convert(Array{Int64,1}, round.(i_values))
i_round_values[i_round_values .< 1] = 1
i_round_values[i_round_values .> 4] = 4
confusmat(4, Array(a_values), i_round_values)

0.5017569337292377, 0.9156282820948355
0.5484085794332431, 0.9583199375373168
0.4446864007274642, 0.9072004594510518
0.2816790692799497, 0.7910072319463368


LoadError: [91mBoundsError: attempt to access 4×4 Array{Int64,2} at index [-1, 1][39m

In [82]:
sum(abs.(diff)/size(diff, 1))

0.6525435891102551

In [47]:

push!(Xs, X)
push!(Ys, Y)

2-element Array{Any,1}:
 [-1.50386 0.857081 … -0.460383 1.00828; 2.32273 -0.900101 … 0.431367 -1.1117; … ; 2.07582 -0.188546 … 0.0321098 0.235293; 3.11935 0.552052 … 2.05196 1.9257]
 [-1.50386 0.857081 … -0.460383 1.00828; 2.32273 -0.900101 … 0.431367 -1.1117; … ; 2.07582 -0.188546 … 0.0321098 0.235293; 3.11935 0.552052 … 2.05196 1.9257]

In [25]:
function roc(i, true_data, column_indices)
    approx = Xs[i].'*Ys[i]
    i_values = approx[:, column_indices][true_data[:, column_indices] .!= 0]
    a_values = true_data[:, column_indices][true_data[:, column_indices] .!= 0]
    
    roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
    tp = sum(map(x -> x[2] > 0, roc_values))
    fp = sum(map(x -> x[2] < 0, roc_values))
    tn = 0
    fn = 0
    sensitivity = Array(Float64, 0)
    specificity = Array(Float64, 0)
    cutoffs = Array(Float64, 0)
    push!(sensitivity, 1)
    push!(specificity, 0)
    push!(cutoffs, 0)
    for v=roc_values
        if v[2] > 0 
            tp -=1
            fn += 1
        elseif v[2] < 0
            fp -= 1
            tn += 1
        end
        if (abs(tp/(tp+fn) - sensitivity[end]) > .01) || (abs(tn/(tn+fp) - specificity[end]) > .01)
            push!(sensitivity, tp/(tp+fn))
            push!(specificity, tn/(tn+fp))
            push!(cutoffs, v[1])
        end
    end
    push!(sensitivity, 0)
    push!(specificity, 1)
    return sensitivity, specificity, cutoffs
end

roc (generic function with 1 method)

In [26]:
# Baseline
baseline_impute = ones(m, 1)*sum(train_data, 1)

function baseline(column_indices)
    i_values = baseline_impute[:, column_indices][validate_data[:, column_indices] .!= 0]
    a_values = validate_data[:, column_indices][validate_data[:, column_indices] .!= 0]
  
    roc_values = sort(collect(filter(x -> (x[2] != 0 && x[1] != 0), zip(i_values, a_values))))
    tp = sum(map(x -> (x[1] > 0 && x[2] > 0), roc_values))
    fp = sum(map(x -> (x[1] > 0 && x[2] < 0), roc_values))
    tn = sum(map(x -> (x[1] < 0 && x[2] < 0), roc_values))
    fn = sum(map(x -> (x[1] < 0 && x[2] > 0), roc_values))

    baseline_sensitivity = tp/(tp+fn)
    baseline_specificity = tn/(tn+fp)
    return baseline_sensitivity, baseline_specificity
end
baseline_sensitivity, baseline_specificity = baseline(1:n)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] filter(::Function, ::Base.Iterators.Zip2{Array{Float64,1},SparseVector{Int64,Int64}}) at ./deprecated.jl:57
 [3] baseline(::UnitRange{Int64}) at ./In[26]:8
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:174
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kelley/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:496
 [8] eventloop(::ZMQ.Socket) at /Users/kelley/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[26], in expression starting on line 18


(0.8265536451829796, 0.6323601464103524)

In [48]:
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity

sensitivity, specificity, cutoffs = roc(1, train_data, 1:n)
plot(1-specificity, sensitivity, linewidth=2, 
    title="Within Instrument Imputation ROC", ylabel="Sensitivity", xlabel="1-Specificity", linewidth=2, label="Train", primary=true,
    xticks = 0:.2:1, yticks = 0:.2:1, size=(500, 400), legend=:right)

sensitivity, specificity, cutoffs = roc(1, validate_data, 1:n)
plot!(1-specificity, sensitivity, linewidth=2, 
    linewidth=2, label="Validate", primary=true)

sensitivity, specificity, cutoffs = roc(2, validate_data, 1:n)
plot!(1-specificity, sensitivity, linewidth=2, 
    linewidth=2, label="Quad Loss", primary=true)

plot!([1-baseline_specificity], [baseline_sensitivity], 
    markersize=5, markershape = :hexagon, primary=true, label="Median Impute")


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] filter(::Function, ::Base.Iterators.Zip2{Array{Float64,1},SparseVector{Int64,Int64}}) at ./deprecated.jl:57
 [3] roc(::Int64, ::SparseMatrixCSC{Int64,Int64}, ::UnitRange{Int64}) at ./In[25]:6
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:174
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kelley/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:496
 [8] eventloop(::ZMQ.Socket) at /Users/kelley/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[48], in expression starting on line 4
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:57
 [3

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] filter(::Function, ::Base.Iterators.Zip2{Array{Float64,1},SparseVector{Int64,Int64}}) at ./deprecated.jl:57
 [3] roc(::Int64, ::SparseMatrixCSC{Int64,Int64}, ::UnitRange{Int64}) at ./In[25]:6
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:174
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kelley/.julia/v0.6/IJulia/src/execute_request.jl:154
 [7] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:496
 [8] eventloop(::ZMQ.Socket) at /Users/kelley/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[48], in expression starting on line 13
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:57
 [

In [28]:
approx = Xs[1].'*Ys[1]
print(size(approx))
print(size(validate_data))
i_values = approx[validate_data .!= 0]
a_values = validate_data[validate_data .!= 0]


(14360, 185)(14360, 185)

76341-element SparseVector{Int64,Int64} with 76341 stored entries:
  [1    ]  =  -1
  [2    ]  =  -1
  [3    ]  =  -1
  [4    ]  =  1
  [5    ]  =  -1
  [6    ]  =  -1
  [7    ]  =  -1
  [8    ]  =  -1
  [9    ]  =  -1
  [10   ]  =  1
           ⋮
  [76331]  =  -1
  [76332]  =  -1
  [76333]  =  -1
  [76334]  =  -1
  [76335]  =  -1
  [76336]  =  -1
  [76337]  =  -1
  [76338]  =  -1
  [76339]  =  -1
  [76340]  =  -1
  [76341]  =  -1

In [99]:
# write to file
writecsv("../data/impute_ordhinge_X_offset_pre$(k).csv", X)
writecsv("../data/impute_ordhinge_Y_offset_pre$(k).csv", Y)
#writecsv("../data/impute_ordhinge_train$(k).csv", train_data)
#writecsv("../data/impute_ordhinge_validate$(k).csv", validate_data)

In [88]:
using LowRankModels
import StatsBase: sample, Weights

# test ordistic loss

## generate data
srand(1);
m,n,k = 200,50,3;
kfit = k+1
d = 7; # number of levels
D = n*d;
# coordinates of covariates
X_real = randn(m,k)
# directions of observations
Y_real = randn(k,n)
# centers of measurement
T_real = sqrt(k)*randn(d,n) # notice x^T y has variance k, so this scales the thresholds in the same way
for j=1:n
	T_real[:,j] = sort(T_real[:,j])
end
# variance of measurement
sigmasq = 1

XY = X_real*Y_real;
A = zeros(Int, (m, n))
for i=1:m
	for j=1:n
		wv = Weights(Float64[exp(-(XY[i,j] - T_real[l,j])^2/sigmasq) for l in 1:d])
		l = sample(wv)
		A[i,j] = l
	end
end

# and the model
losses = fill(OrdisticLoss(d),n)
rx, ry = lastentry1(QuadReg(.01)), lastentry_unpenalized(QuadReg(.01));
glrm = GLRM(A,losses,rx,ry,kfit, scale=false, offset=false, X=randn(kfit,m), Y=randn(kfit,D));

# fit w/o initialization
p = Params(1, max_iter=10, abs_tol=0.0000001, min_stepsize=0.000001)
@time X,Y,ch = fit!(glrm, params=p);
XYh = X'*Y[:,1:d:D];
@show ch.objective
println("After fitting, parameters differ from true parameters by $(vecnorm(XY - XYh)/sqrt(prod(size(XY)))) in RMSE")
A_imputed = impute(glrm)
println("After fitting, $(sum(A_imputed .!= A) / prod(size(A))*100)\% of imputed entries are wrong")
println("After fitting, imputed entries are off by $(sum(abs,(A_imputed - A)) / prod(size(A))*100)\% on average")
println("(Picking randomly, $((d-1)/d*100)\% of entries would be wrong.)\n")

# initialize
init_svd!(glrm)
XYh = glrm.X' * glrm.Y[:,1:d:D]
println("After initialization with the svd, parameters differ from true parameters by $(vecnorm(XY - XYh)/sqrt(prod(size(XY)))) in RMSE")
A_imputed = impute(glrm)
println("After initialization with the svd, $(sum(A_imputed .!= A) / prod(size(A))*100)\% of imputed entries are wrong")
println("After fitting, imputed entries are off by $(sum(abs,(A_imputed - A)) / prod(size(A))*100)\% on average")
println("(Picking randomly, $((d-1)/d*100)\% of entries would be wrong.)\n")

# fit w/ initialization
p = Params(1, max_iter=10, abs_tol=0.0000001, min_stepsize=0.000001)
@time X,Y,ch = fit!(glrm, params=p);
XYh = X'*Y[:,1:d:D];
@show ch.objective
println("After fitting, parameters differ from true parameters by $(vecnorm(XY - XYh)/sqrt(prod(size(XY)))) in RMSE")
A_imputed = impute(glrm)
println("After fitting, $(sum(A_imputed .!= A) / prod(size(A))*100)\% of imputed entries are wrong")
println("After fitting, imputed entries are off by $(sum(abs,(A_imputed - A)) / prod(size(A))*100)\% on average")
println("(Picking randomly, $((d-1)/d*100)\% of entries would be wrong.)\n")


Fitting GLRM


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:52
 [5] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at ./<missing>:0
 [6] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /Users/kelley/.julia/v0.6/LowRankModels/src/fit.jl:11
 [7] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM) at ./<missing>:0
 [8] include_string(::String, ::String) at ./loading.jl:522
 [9] include_string(::Module, ::String, ::String) at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:174
 [10] execute

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] exp(::Array{Float64,1}) at ./deprecated.jl:57
 [3] grad(::LowRankModels.OrdisticLoss, ::Array{Float64,1}, ::Int64) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:526
 [4] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:128
 [5] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at ./<missing>:0
 [6] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /Users/kelley/.julia/v0.6/LowRankModels/src/fit.jl:11
 [7] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM) at ./<missing>:0
 [8] include_string(::String, ::String) at ./loading.jl:522
 [9] include_string(::Module, ::String, ::String) at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:174
 [10] execute_r

LoadError: [91mInterruptException:[39m